In [1]:
import tensorflow as tf
import numpy as np
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()
# (m, n) is (# of sampels, # of features)
m, n = housing.data.shape

housing_data_plus_bias = np.c_[np.ones((m,1)), housing.data]
housing_targets = housing.target.reshape(-1,1)

std_scaler = StandardScaler()
scaled_housing_data_plus_bias = std_scaler.fit_transform(housing_data_plus_bias)

In [2]:
# n+1 is the number of features. (x1, ... xn + x0(bias)).
X = tf.placeholder(dtype=tf.float32, shape=(None, n+1), name='X')
y = tf.placeholder(dtype=tf.float32, shape=(None, 1), name='y')
XT = tf.transpose(X)

theta = tf.Variable(tf.random_uniform(shape=[n+1, 1],
                                      minval=-1.0,
                                      maxval=1.0),
                    name='theta')
y_pred = tf.matmul(X, theta, name='predictions')

with tf.name_scope('loss') as scope:
    error = y_pred - y
    mse = tf.reduce_mean(tf.square(error), name='mse')

gradients = tf.gradients(mse, [theta])[0]
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
training_op = optimizer.minimize(mse)

Instructions for updating:
Colocations handled automatically by placer.


In [3]:
def fetch_batch(epoch, batch_index, batch_size):
    np.random.seed(epoch * n_batches + batch_index)
    indices = np.random.randint(m, size=batch_size)
    X_batch = scaled_housing_data_plus_bias[indices]
    y_batch = housing_targets.reshape(-1, 1)[indices]
    return X_batch, y_batch

In [4]:
from datetime import datetime
from os import path
from pathlib import Path

tf_log_root = path.join(Path.home(), 'workspace/machine-learning/tensorboard')
datestamp = datetime.utcnow().strftime('%Y%m%d%H%M%S')
logdir = path.join(tf_log_root, datestamp)

mse_summary = tf.summary.scalar('MSE', mse)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

In [5]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    n_epochs = 100
    batch_size = 100
    n_batches = int(np.ceil(m / batch_size))
    print('batch_size={}, n_batches={}'.format(batch_size, n_batches))

    sess.run(init)
    
    for epoch in range(n_epochs):
        training_errors = []
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            if batch_index % 10 == 0:
                summary_str = mse_summary.eval(feed_dict={X: X_batch, y: y_batch})
                step = epoch * n_batches + batch_index
                file_writer.add_summary(summary_str, step)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        best_theta = theta.eval()

batch_size=100, n_batches=207


In [6]:
file_writer.close()